In [1]:
import numpy as np
import csv
import re
import math

In [2]:
#creating vocabulary list
vocab = list()
ap1 = list()
m=0 #number of data examples/tweets
stop_words = ['a', 'an', 'the', 'it','i','to','in','my','am','is','if','for','be','and','im' ,'you','herself', 'it', 'its', 'been', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'of', 'at', 'by', 'for', 'with', 'to', 'from']
exclude= ['-','@','#','',':',';','...','.','..','!','/','']
tweet = 0

with open('/Users/syedhadi/Desktop/Assignment3/Tweets.csv') as file:
    iterator = csv.reader(file, delimiter=',')
    for text in iterator:
        if tweet == 0:
            tweet += 1
        else:
            st1 = text[1].lower()
            st1=st1.replace('@','')
            st1=st1.replace('#','')
            st1=st1.replace('$','')
            st1=st1.replace(':','')
            st1=st1.replace(';','')
            st1=st1.replace('/','')
            st1=st1.replace('.','')
            st1=st1.replace('&','')
            st1=st1.replace('-','')
            st1=st1.replace('=','')
            st1=st1.replace('!','')
            st1=st1.replace('?','')
            st1=st1.replace(',','')
            st1=st1.replace('"','')
            st1=st1.replace('~','')
            st1=st1.replace('(','')
            st1=st1.replace(')','')
            st1=st1.replace('+','')
            st1=st1.replace('*','')
            st1=st1.replace('^','')
            st1=st1.replace('“','')
            st1=st1.replace('‘','')
            st1=st1.replace("'","")
            
            
            words = st1.split ()
            for i in words:
                if i in stop_words or i.isnumeric() or i in exclude:
                    continue
                if i in ap1:
                    if i in vocab:
                        continue
                    else:
                        vocab.append(i)
                else:
                    ap1.append(i)
                        
                        
                    
            tweet += 1
    

In [3]:
n = len(vocab) #number of features
m = tweet-1  #trainng exaples count

Y = list() #sentiment
X = np.zeros((m,n)) #features matrix

In [4]:
#extracting into variables 
with open('/Users/syedhadi/Desktop/Assignment3/Tweets.csv') as file:
    iterator = csv.reader(file, delimiter=',')
    tweet = 0
    for text in iterator:
        if tweet == 0:
            tweet += 1
        else:
            if(text[0]=='positive'):
                Y.append(1)
            if(text[0]=='negative'):
                Y.append(2)
            if(text[0]=='neutral'):
                Y.append(0)
                
            st1 = text[1].lower()
            st1=st1.replace('@','')
            st1=st1.replace('#','')
            st1=st1.replace('$','')
            st1=st1.replace(':','')
            st1=st1.replace(';','')
            st1=st1.replace('/','')
            st1=st1.replace('.','')
            st1=st1.replace('&','')
            st1=st1.replace('-','')
            st1=st1.replace('=','')
            st1=st1.replace('!','')
            st1=st1.replace('?','')
            st1=st1.replace(',','')
            st1=st1.replace('"','')
            st1=st1.replace('~','')
            st1=st1.replace('(','')
            st1=st1.replace(')','')
            st1=st1.replace('+','')
            st1=st1.replace('*','')
            st1=st1.replace('^','')
            st1=st1.replace('“','')
            st1=st1.replace('‘','')
            st1=st1.replace("'","")
            words = st1.split()
            
            for i in words:
                if i in vocab:
                    index = vocab.index(i)
                    X[tweet-1][index] += 1
                    
                    
            tweet += 1

In [5]:
# STRATIFICATION 

n = len(Y)
X0= np.ones((n,1))
X = np.hstack((X0, X))
Y= np.asarray(Y)
Y = Y.reshape(Y.size,1)

X = np.hstack((X,Y))


np.random.shuffle(X)



In [6]:
#TRAIN TEST SPLIT
s= int(0.8*Y.size) #train data split
r= (X[1,:].size)-2 # labels split index



XTR = X[0:s,0:r].copy()
YTR = X[0:s,r+1].copy()

XTE = X[s:Y.size, 0:r].copy()
YTE = X[s:Y.size, r+1].copy()

In [7]:
#hyperparameters

T0 = (np.zeros(XTR[1,:].size))
T1 = np.zeros(XTR[1,:].size)
T2 = np.zeros(XTR[1,:].size)

In [8]:
#creating functions here 

def softmax(X,T0, T1, T2):
    
    z0 = np.dot(X,T0)
    z1 = np.dot(X,T1)
    z2 = np.dot(X,T2)  
    
    den = np.exp(z0)+ np.exp(z1) + np.exp(z2)  
    
    Z= np.zeros(3)    
    
    Z[0]= np.exp(z0)/den #probability of being neutral
    Z[1]= np.exp(z1)/den #probability of being positive
    Z[2]= np.exp(z2)/den #probability of being negative 
    
    return (Z)

    
#cost function 

def Cost(XTR ,T0, T1, T2, YTR):
    m= YTR.size
    soft = np.zeros(3)
    L = np.zeros(m)
    L0=0
    L1=0
    L2=0
    for i in range (m):
        soft = softmax(XTR[i],T0,T1,T2)   
        L0 = (np.ndarray.tolist(soft).index(max(soft))== YTR[i])*np.log(soft[0])
        L1 = (np.ndarray.tolist(soft).index(max(soft))== YTR[i])*np.log(soft[1])
        L2 = (np.ndarray.tolist(soft).index(max(soft))== YTR[i])*np.log(soft[2])   
        L[i] = -1*(L0+L1+L2)
    
    return (np.mean(L))

#mini batch gradient descent

def MGD(XTR, YTR, lr, iters):
    
    m       = YTR.size #trainnig data size
    n       = XTR[1,:].size # number of features 
    Params0 = np.zeros(n)
    Ptemp0  = np.zeros(n)
    Params1 = np.zeros(n)
    Ptemp1  = np.zeros(n)
    Params2 = np.zeros(n)
    Ptemp2  = np.zeros(n)

    for epoch in range (iters):
    
        for j in range (365):
            Sum0 = np.zeros(n)
            Sum1 = np.zeros(n)
            Sum2 = np.zeros(n)
            s0=0
            s1=0
            s2=0
            inc = j*32
            for i in range (32):
                i += inc
                soft = softmax(XTR[i,:],Params0,Params1,Params2)
                #print(soft)
                if (YTR[i]==0): #nuetral 
                    Sum0 += -(1 - np.log(soft[0])) * XTR[i,:]
                    s0+=1
                if (YTR[i]==1): #positive
                    Sum1 += -(1 - np.log(soft[1])) * XTR[i,:]
                    s1+=1
                if YTR[i]==2: #negative
                    Sum2 += -(1 - np.log(soft[2])) * XTR[i,:]
                    s2+=1
            if s0 == 0:
                s0+=1        
            GradVector0 = (lr/s0) * Sum0
            if s1 == 0:
                s1+=1
            GradVector1 = (lr/s1) * Sum1
            if s2 == 0:
                s2+=1
            GradVector2 = (lr/s2) * Sum2
            Ptemp0 = Params0 - GradVector0
            Params0 = Ptemp0
            Ptemp1 = Params1 - GradVector1
            Params1 = Ptemp1
            Ptemp2 = Params2 - GradVector2
            Params2 = Ptemp2
    
    return Params0,Params1, Params2

    
T = MGD(XTR,YTR,0.01,30)

In [9]:
#storing new hyper parameters
T0= T[0]
T1= T[1]
T2= T[2]

In [10]:
#Macro ACCURACY CALCULATED HERE

n = YTE.size
t=0
for i in range (n):
    a=softmax(XTE[i,:],T0,T1,T2)
    if (np.ndarray.tolist(a).index(max(a)) == YTE[i]):
        t+=1

Accuracy = t/n

In [11]:
#Measures Calculated here 
tp0 = 0
fp0 = 0
tn0 = 0
fn0 = 0

tp1 = 0
fp1 = 0
tn1 = 0
fn1 = 0

tp2 = 0
fp2 = 0
tn2 = 0
fn2 = 0
c=0

for i in range (n):
    a=softmax(XTE[i,:],T0,T1,T2)
    pred = np.ndarray.tolist(a).index(max(a))
    if (pred == YTE[i]):
        if YTE[i]==0:
            c+=1
            tp0+=1 ###
            tn1+=1
            tn2+=1
        if YTE[i]==1:
            c+=1
            tp1+=1 ###
            tn0+=1
            tn2+=1
        if YTE[i]==2:
            c+=1
            tp2+=1 ###
            tn0+=1
            tn1+=1
    if (pred != YTE[i]):
        if YTE[i]==0:
            fn0+=1
            if pred==1:
                c+=1
                fp1+=1 
                tn2+=1
            if pred==2:
                c+=1
                fp2+=1
                tn1+=1
                
        if YTE[i]==1:
            fn1+=1
            if pred==0:
                c+=1
                fp0+=1 
                tn2+=1
            if pred==2:
                c+=1
                fp2+=1
                tn0+=1
        if YTE[i]==2:
            fn2+=1
            if pred==0:
                c+=1
                fp0+=1 
                tn1+=1
            if pred==1:
                c+=1
                fp1+=1
                tn0+=1

In [28]:
#MACROAVERAGE MEASURES
#Precision 
P0 = tp0/(fp0+tp0)
P1 = tp1/(fp1+tp1)
P2 = tp2/(fp2+tp2)

#Recall
R0 = tp0/(tp0+fn0)
R1 = tp1/(tp1+fn1)
R2 = tp2/(tp2+fn2)

#F-1 Measure
F1_0 = (2*R0*P0)/(R0+P0)
F1_1 = (2*R1*P1)/(R1+P1)
F1_2 = (2*R2*P2)/(R2+P2)

F1 = (F1_0+F1_1+F1_2)/3

print('Macro Average Measures :')
print('Precision = ', math.ceil(((P0+P1+P2)/3)*100),'%')
print('Recall = ', math.ceil(((R0+R1+R2)/3)*100),'%')
print('Accuracy = ', math.ceil(Accuracy*100),'%')
print('F1- Measure = ', math.ceil(F1*100),'%' )

Macro Average Measures :
Precision =  68 %
Recall =  72 %
Accuracy =  74 %
F1- Measure =  69 %


In [36]:
#MicroAverage Measures 
tps=tp0+tp1+tp2
fps=fp0+fp1+fp2
fns=fn0+fn1+fn2

Pr = (tps)/(tps+fps)
Re = (tps)/(tps+fns)

MicF1 = (2*Pr*Re)/(Pr+Re)

print('Micro Average Measures :')
print('Precision = ', math.ceil((Pr)*100),'%')
print('Recall = ', math.ceil((Re)*100),'%')
print('Accuracy = ', math.ceil((Re)*100),'%')
print('F1- Measure = ', math.ceil(MicF1*100),'%' )

Micro Average Measures :
Precision =  74 %
Recall =  74 %
Accuracy =  74 %
F1- Measure =  74 %
